In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import integrate
import geopy.distance
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import requests
import time

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [2]:
cred = credentials.Certificate('./../Backend/serviceAccount.json')
app_firebase = firebase_admin.initialize_app(cred)
db = firestore.client()

In [3]:
import sys
 
sys.path.append('./../Backend/')

In [4]:

from key_value.token import apiKey, apiUrl, auth_header


 


In [5]:
auth_header

{'x-apikey': 'IaF6zOUaj8VG4ambmUwOodEFaxBnrR3A'}

In [15]:
path = './ONLY_TEST/'
csv_files = glob.glob(os.path.join(path, "*.csv"))
for f in csv_files:
    # print(f)
    if os.name == 'nt':
        filename = f.split('\\')[1]
    else:
        filename = f.split('/')[2]
        
    flight_id = filename.split('-')[0]
    # print(flight_id)
    doc_ref = db.collection('aircraft_engine').document(f'{flight_id}')
    docs = doc_ref.get()

    if docs.exists:
        print("Found")
        # print(docs.to_dict())
    else:
        print("Not found", flight_id)
        #----- get aircraft_type from flight_ID -----#
        response = requests.get(apiUrl + f"flights/{flight_id}",headers=auth_header)

        if response.status_code == 200:
            # print("got aircraft_type")
            aircraft_type = response.json()['flights'][0]['aircraft_type']
            
            #----- get engine data from aircraft_type -----#
            response = requests.get(apiUrl + f"aircraft/types/{aircraft_type}",headers=auth_header)
            
            if response.status_code == 200:
                # print("got engine data")
                engine_data = response.json()
                engine_data.pop('manufacturer', None)
                engine_data.pop('description', None)
                engine_data['flight_id'] = flight_id
                
                #----- add to firestore -----#
                doc_ref = db.collection(f'aircraft_engine').document(f'{flight_id}')
                doc_ref.set(engine_data)
                
            else:
                print("Error executing request")
            
        else:
            print("Error executing request")
        
        type_engine = engine_data['type']
        docs_engine = db.collection(f'NPD_ID').document(f'{type_engine}').get()
        if not docs_engine.exists:
            print(f'engine of {flight_id} not found\n', type_engine)


        time.sleep(10)
    




Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Not found NOK532
Found
Not found NOK578
Found
Found
Found
Found


# read NPD_ID

In [16]:
path = './ONLY_TEST/'
csv_files = glob.glob(os.path.join(path, "*.csv"))
for f in csv_files:
    # print(f)
    if os.name == 'nt':
        filename = f.split('\\')[1]
    else:
        filename = f.split('/')[2]
        
    flight_id = filename.split('-')[0]
    # print(flight_id)
    
    #----- find ACFT_ID -----#
    acft_ref = db.collection(f'aircraft_engine').document(f'{flight_id}')
    acft_docs = acft_ref.get()

    if acft_docs.exists:
        # print("Found")
        acft_docs = acft_docs.to_dict()
        ACFT_ID = acft_docs['type']
        # print(ACFT_ID)
    else:
        print("Not found",flight_id)
        continue
    
    #----- find NPD_ID -----#
    npd_ref = db.collection(f'NPD_ID').document(f'{ACFT_ID}')
    npd_docs = npd_ref.get()
    
    if npd_docs.exists:
        # print("Found")
        npd_docs = npd_docs.to_dict()
        NPD_ID = npd_docs['NPD_ID']
        # print(NPD_ID)
    else:
        print("----------------Not found------------",flight_id)
        continue
    
    
